## 1. Understanding Data

In [ ]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

### Before Start
본격적으로 시작하기 전에 데이터에 대해서 아주 약간만 깊이 있게 이해해보는 시간을 가져보려고 합니다.

### Q. 디스크의 CSV파일의 용량은 그렇게 높진 않은데 메모리로 읽기만 하면 몇 배로 늘어나는 이유는?
캐글을 하시다 보면 이런 경험이 한번쯤은 다들 있으실 것 같습니다.   
분명히 CSV 파일로는 1GB 보다 아래였는데 판다스로 read를 하면 2~3GB로 늘어나는 경우가 종종 있는데, 이유가 무엇일까요?

### Load data

In [ ]:
PATH = '../input/kakr-4th-competition/'
train = pd.read_csv(PATH + 'train.csv')
test  = pd.read_csv(PATH + 'test.csv')

### 데이터 확인

* id
* age : 나이
* workclass : 고용 형태
* fnlwgt : 사람 대표성을 나타내는 가중치 (final weight의 약자)
* education : 교육 수준
* education_num : 교육 수준 수치
* marital_status: 결혼 상태
* occupation : 업종
* relationship : 가족 관계
* race : 인종
* sex : 성별
* capital_gain : 양도 소득
* capital_loss : 양도 손실
* hours_per_week : 주당 근무 시간
* native_country : 국적
* income : 수익 (예측해야 하는 값)
    * \>50K : 1
    * <=50K : 0

# 데이터 확인

In [ ]:
train.head()

In [ ]:
# target에 해당하는 컬럼을 바로 label로 지정하고, train_data_set에서는 제외하는 것도 방법
label = train['income']

del train['income']

train

In [ ]:
# lable에 대해 범위를 지정해서 값 변경, 대회 규칙에 따라서
label = label.map(lambda x : 1 if x== '>50K' else 0) # True/False 값을 int로 변형해서 저장

label

In [ ]:
# id 컬럼은 필요 없는 컬럼이기 때문에 삭제

del train['id']
del test['id']

In [ ]:
# 각 컬럼들에 대한 정보 확인

train.info()

In [ ]:
# 각 컬럼들 중 수치 값을 갖는 컬럼에 대한 통계적 수치 정보 확인

train.describe()

# 결측치 처리

* workclass, occupation, native_country 컬럼에 결측치가 존재
* '?'로 표시된 데이터는 해당 컬럼의 최빈값으로 결측치를 처리


* 일반적으로는 최빈값을 통해 대체하는 경우가 많지만,
* 만약 다른 컬럼을 통해서 처리할 수 있다면 그렇게 하는 것이 더 좋은 방법
* ex) education_num 등


* 정답은 없고, 해당 데이터의 상태를 보고서 결정해야 함

In [ ]:
has_na_colums = ['workclass', 'occupation', 'native_country']

for c in has_na_colums :
    # mode() 해당 컬럼의 최빈 값을 가져오는 함수
    train.loc[train[c] == '?', c]  = train[c].mode()[0]
    test.loc[test[c] == '?', c]  = test[c].mode()[0]

In [ ]:
# 적용 후 결측치 처리가 된건지 확인
(train[has_na_colums] == '?').sum()

# Log 변환

In [ ]:
train['capital_gain'].plot.hist()

In [ ]:
# 치우친 정도를 완화시키기 위해 log 변환 적용

train['log_capital_gain'] = train['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
test['log_capital_gain'] = test['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)

In [ ]:
# 적용 후 확인

train['log_capital_gain'].plot.hist()

In [ ]:
# 이제 원래 컬럼은 필요 없다고 판단하고 삭제

train = train.drop(columns=['capital_gain'])
test = test.drop(columns=['capital_gain'])

In [ ]:
# capital_loss 컬럼에도 적용

train['log_capital_loss'] = train['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)
test['log_capital_loss'] = test['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)

train = train.drop(columns=['capital_loss'])
test = test.drop(columns=['capital_loss'])

train.head()

# 데이터 쪼개기, Split

* Train Data : 모델을 학습하는 데 사용하는 데이터(모델이 알고 있는 학습할 데이터, 과거 데이터)
* Valid Data : 학습한 모델의 성능을 검증하는 데이터(모델이 모르는 학습하지 않을 데이터, 과거 데이터)
* Test Data : 학습한 모델로 예측할 데이터(모델이 모르는 데이터, 미래 데이터)

* train_test_split
    * test_size : Valid(test)의 크기 비율 지정
    * random_state : 데이터 쪼갤 때 내부적으로 사용하는 난수 값(default는 매번 다름)
    * shuffle : 데이터 쪼갤 때 섞을지 유무 지정
    * stratify : 쪼개기 이전의 클래스 비율을 쪼개고 난 후에도 유지하기 위해 설정하는 값

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
    train, label,
    test_size = 0.3,
    random_state = 2021,
    shuffle = True,
    stratify = label
)

In [ ]:
x_train.head()

In [ ]:
# 인덱스 초기화
x_train = x_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)

test = test.reset_index(drop=True)

In [ ]:
x_train.head()

# 스케일링, Scaling

In [ ]:
# Scikit-learn 라이브러리에 있는 Standard Scaler 사용해서 수치형 변수들의 표준화 진행

ctgy_col = [c for c, t in zip(x_train.dtypes.index, x_train.dtypes) if t == 'O'] # categorical columns
numr_col = [c for c in x_train.columns if c not in ctgy_col] # numerical columns

print(ctgy_col)
print(numr_col)

In [ ]:
# Scaling과 같은 전처리는 항상 train data를 기준으로 진행

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train[numr_col] = scaler.fit_transform(x_train[numr_col])

# 여기서 valid와 test는 fit하면 안됨
x_valid[numr_col] = scaler.transform(x_valid[numr_col])
test[numr_col] = scaler.transform(test[numr_col])

In [ ]:
x_train.describe()

# 인코딩, Encoding

In [ ]:
# 범주형 변수는 Onehot Encoding 적용

from sklearn.preprocessing import OneHotEncoder

tmp_all = pd.concat([x_train, x_valid, test])

ohe = OneHotEncoder(sparse=False)
ohe.fit(tmp_all[ctgy_col])

In [ ]:
ohe_col = list()

for lst in ohe.categories_ :
    ohe_col += lst.tolist()

In [ ]:
new_train_ctgy = pd.DataFrame(ohe.transform(x_train[ctgy_col]), columns=ohe_col)
new_valid_ctgy = pd.DataFrame(ohe.transform(x_valid[ctgy_col]), columns=ohe_col)
new_test_ctgy = pd.DataFrame(ohe.transform(test[ctgy_col]), columns=ohe_col)

In [ ]:
new_train_ctgy.head()

In [ ]:
# 인코딩 적용
x_train = pd.concat([x_train, new_train_ctgy], axis=1)
x_valid = pd.concat([x_valid, new_valid_ctgy], axis=1)
test = pd.concat([test, new_test_ctgy], axis=1)

# 기존 컬럼 제거
x_train = x_train.drop(columns = ctgy_col)
x_valid = x_valid.drop(columns = ctgy_col)
test = test.drop(columns = ctgy_col)

In [ ]:
x_train.head()

# Scikit-learn 기반 분류 모델

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
# 로지스틱 회귀 모델

lr = LogisticRegression()

lr.fit(x_train, y_train)

y_pred = lr.predict(x_valid)

f1_score(y_valid, y_pred, average='micro')

In [ ]:
# 서포트 벡터 머신(rbf 커널)

svm = SVC()

svm.fit(x_train, y_train)

y_pred = svm.predict(x_valid)

f1_score(y_valid, y_pred, average='micro')

In [ ]:
# 랜덤 포레스트

rf = RandomForestClassifier()

rf.fit(x_train, y_train)

y_pred = rf.predict(x_valid)

f1_score(y_valid, y_pred, average='micro')

In [ ]:
# XGB

xgb = XGBClassifier()

xgb.fit(x_train, y_train)

y_pred = xgb.predict(x_valid)

f1_score(y_valid, y_pred, average='micro')

In [ ]:
# LGBM

lgbm = LGBMClassifier()

lgbm.fit(x_train, y_train)

y_pred = lgbm.predict(x_valid)

f1_score(y_valid, y_pred, average='micro')

# 3. k-Fold Cross Validation

In [ ]:
# preprocess 진행한 data 확인
x_train.head()

In [ ]:
x_valid.head()

In [ ]:
test.shape

In [ ]:
train = pd.read_csv("/kaggle/input/kakr-4th-competition/train.csv")
label = train['income']

del train['income']

test = pd.read_csv("/kaggle/input/kakr-4th-competition/test.csv")

has_na_columns = ['workclass', 'occupation', 'native_country']

from sklearn.preprocessing import StandardScaler

cat_columns = [c for c, t in zip(train.dtypes.index, train.dtypes) if t == 'O'] 
num_columns = [c for c in train.dtypes.index if c not in cat_columns]

print('범주형 변수: \n{}\n\n 수치형 변수: \n{}\n'.format(cat_columns, num_columns))

In [ ]:
def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train = tmp_x_train.reset_index(drop=True)
    tmp_x_valid = tmp_x_valid.reset_index(drop=True)
    tmp_x_test  = tmp_x_test.reset_index(drop=True)
    
    for c in has_na_columns:
        tmp_x_train.loc[tmp_x_train[c] == '?', c] = tmp_x_train[c].mode()[0]
        tmp_x_valid.loc[tmp_x_valid[c] == '?', c] = tmp_x_valid[c].mode()[0]
        tmp_x_test.loc[tmp_x_test[c]   == '?', c] = tmp_x_test[c].mode()[0]
    
    tmp_x_train['log_capital_loss'] = tmp_x_train['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)
    tmp_x_valid['log_capital_loss'] = tmp_x_valid['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)
    tmp_x_test['log_capital_loss'] = tmp_x_test['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)
    
    tmp_x_train['log_capital_gain'] = tmp_x_train['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
    tmp_x_valid['log_capital_gain'] = tmp_x_valid['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
    tmp_x_test['log_capital_gain'] = tmp_x_test['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
    
    tmp_x_train = tmp_x_train.drop(columns=['capital_loss', 'capital_gain'])
    tmp_x_valid = tmp_x_valid.drop(columns=['capital_loss', 'capital_gain'])
    tmp_x_test  = tmp_x_test.drop(columns=['capital_loss', 'capital_gain'])
    
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])
    
    tmp_all = pd.concat([tmp_x_train, tmp_x_valid, tmp_x_test])

    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_all[cat_columns])
    
    ohe_columns = list()
    for lst in ohe.categories_:
        ohe_columns += lst.tolist()
    
    tmp_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]), columns=ohe_columns)
    tmp_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]), columns=ohe_columns)
    tmp_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]), columns=ohe_columns)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_valid_cat], axis=1)
    tmp_x_test = pd.concat([tmp_x_test, tmp_test_cat], axis=1)

    tmp_x_train = tmp_x_train.drop(columns=cat_columns)
    tmp_x_valid = tmp_x_valid.drop(columns=cat_columns)
    tmp_x_test = tmp_x_test.drop(columns=cat_columns)
    
    return tmp_x_train.values, tmp_x_valid.values, tmp_x_test.values

In [ ]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5

skf = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = 2021)

In [ ]:
def xgb_f1(y, t, treshold=0.5) : 
    t = t.get_label()
    y_bin = (y > threshold).astype(int)
    return 'f1', f1_score(t, y_bin, averager='micro')

In [ ]:
val_scores = list()

for i, (trn_idx, val_idx) in enumerate(skf.split(train, label)) : 
    x_train, y_train = train.iloc[trn_idx, :], label[trn_idx]
    x_valid, y_valid = train.iloc[val_idx, :], label[val_idx]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    clf = XGBClassifier(tree_method='gpu_hist')
    
    # 모델 학습
    clf.fit(
        x_train, y_train,
        eval_set = [[x_valid, y_valid]],
        eval_metric = xgb_f1,
        early_stopping_rounds = 100,
        verbose = 100
    )
    
    # 훈련, 검증 데이터 Log Loss 확인
    trn_f1_score = f1_score(y_train, clf.predict(x_train), average='micro')
    val_f1_score = f1_score(y_valid, clf.predict(x_valid), average='micro')
    print('{} Fold, train f1_score : {:.4f}, validation f1_score : {:.4f}\n'. format(i, trn_f1_score, val_f1_score))
    
    val_scores.append(val_f1_score)
    
# 교차 검증 F1 Score 평균 계산
print('Cross Validation Score : {:.4f}'. format(np.mean(val_scores)))

# OOF

In [ ]:
val_scores = list()
oof_pred = np.zeros((test.shape[0], )) #

for i, (trn_idx, val_idx) in enumerate(skf.split(train, label)):
    x_train, y_train = train.iloc[trn_idx, :], label[trn_idx]
    x_valid, y_valid = train.iloc[val_idx, :], label[val_idx]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    clf = XGBClassifier(tree_method='gpu_hist')
    
    # 모델 학습
    clf.fit(x_train, y_train,
            eval_set = [[x_valid, y_valid]], 
            eval_metric = xgb_f1,        
            early_stopping_rounds = 100,
            verbose = 100,  )

    # 훈련, 검증 데이터 F1 Score 확인
    trn_f1_score = f1_score(y_train, clf.predict(x_train), average='micro')
    val_f1_score = f1_score(y_valid, clf.predict(x_valid), average='micro')
    print('{} Fold, train f1_score : {:.4f}4, validation f1_score : {:.4f}\n'.format(i, trn_f1_score, val_f1_score))
    
    val_scores.append(val_f1_score)
    
    oof_pred += clf.predict_proba(x_test)[: , 1] / n_splits #
    

# 교차 검증 F1 Score 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

# STACKING